# Neural Data

Check and explore the neural data for a single-neuron project,

This notebook covers loading data, preparing metadata, getting micro-LFP data, and splitting channels ready for sorting.

In [ ]:
# Add local folder with `sort` module
import sys
sys.path.append('..')
from sort import Paths
from sort.io import load_object, save_to_h5file
from sort.nsp import load_blackrock, check_blackrock_file_info
from sort.channels import make_split_file_name

## Paths & Files

In [ ]:
# Set the project path
project_path = '/path/'

In [ ]:
# Get a list of subjects
subjects = Paths(project_path).get_files('recordings')

## Settings

In [ ]:
# Set session information
SESSION = {
    'experiment' : 'XX',
    'subject' : 'XX',
    'session' : 0,
}

In [ ]:
# Create database structure for the current file
paths = Paths(project_path, **SESSION)

In [ ]:
# Notebook settings
SAVE_OUT = False

In [ ]:
# Check session that is being loaded
SESSION

## Site Specific Settings

If there is any site specific settings (for example, which files to load) these go here.

In [ ]:
...

## Check Available Data Files

In [ ]:
# Get the list of available neural data files
data_files = data_paths.get_files('neural')
data_files

In [ ]:
# Check available files by specific type
nsp_files = paths.get_files('neural', select='XX')
nsp_files

In [ ]:
# Select data file to process
data_file = nsp_files[0]
data_file

## Load Electrodes Metadata File

In [ ]:
# Load the electrodes object
electrodes = load_object(SESSION['subject'] + '.electrodes', paths.electrodes)

In [ ]:
# Convert and check the dataframe representation of the electrodes
edf = electrodes.to_dataframe()
edf.head()

## Load Data Files

In [ ]:
# Example: load files from blackrock
reader = load_blackrock(data_file, paths.neural, ...)

### Check Loaded Data

In [ ]:
...

## Channels (neural file)

Check any channel information available in the NSP file(s).

In [ ]:
...

### Check Microwire Channel Information

In [ ]:
# Get the number of micro channels
n_micros = len(micro_chans)

# Validate the expected number of channels
assert len(probes) * 8 == n_micros

# Check the number of probes / channels
print('Data: {} probes ({} channels)'.format(len(probes), len(micro_chans)))

## Neural Data

In [ ]:
# Set block and segment of interest
block_ind = 0
seg_ind = 0

### Check example data segment

In [ ]:
# Extract an example data chunk - blackrock example
data_chunk = reader.get_analogsignal_chunk(block_ind, seg_ind, 10*fs, 11*fs, channel_indexes=chi).T

## Split Channel Files

Save out extracted channel data.

In [ ]:
if SAVE_OUT:
    
    print('Extracting channel data for {}...'.format(SESSION['subject']))
    for ch_name in micro_chans:
                
        channel_data = {
            'data' : np.squeeze(reader.get_analogsignal_chunk(block_ind, seg_ind, channel_names=[ch_name])),
            'sr' : fs, 
        }
        
        split_file_name = make_split_file_name(ch_name, site, edf)
        save_to_h5file(channel_data, split_file_name, paths.sorting, ext='.hdf5')
        print('\tsaved channel: {}'.format(split_file_name))